# Kaggle 필사 노트북
## Titanic - Machine Learning from Disaster

이 노트북은 Kaggle Titanic 대회의 필사 학습을 위해 작성되었습니다.  
Google Drive + Kaggle API + GitHub 연동 기반입니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Kaggle API 설정
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle_replica/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# titanic 데이터 다운로드

# !kaggle competitions download -c titanic
# !unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# target_dir = "/content/drive/MyDrive/kaggle_replica/titanic"

# 파일 이동
# !mv train.csv {target_dir}/
# !mv test.csv {target_dir}/
# !mv gender_submission.csv {target_dir}/

In [3]:
import numpy as np
import pandas as pd
# 이 부분은 pd.Series(...)로 쓰는 것이 오히려 가독성 유지에 도움이 된다.
from pandas import Series
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
sns.set_context('notebook', font_scale=1.2)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [4]:
# import pandas as pd

train_path = '/content/drive/MyDrive/kaggle_replica/titanic/train.csv'
test_path = '/content/drive/MyDrive/kaggle_replica/titanic/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)